In [1]:
from torch.utils.data import DataLoader, TensorDataset
import wandb

from m2_utilities.load_data import load_trajectories
from m2_utilities.preprocessor import scale, Preprocessor
from m2_utilities.qwen import load_qwen, train
from m2_utilities.lora import apply_lora
from m2_utilities.flops import compute_flops

%load_ext autoreload
%autoreload 2

In [2]:
model, tokenizer = load_qwen()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [3]:
LORA_RANK = 4
apply_lora(model, LORA_RANK)

### Preprocessing Trajectories to Tokens

In [4]:
# Load
trajectories = load_trajectories("data/lotka_volterra_data.h5")

# Split and scale
train_trajectories = scale(trajectories[:700])
val_trajectories = scale(trajectories[700:850])

# Encode
DECIMALS = 3
preprocessor = Preprocessor(DECIMALS)
train_ids = preprocessor.encode(train_trajectories, chunk=True)
val_ids = preprocessor.encode(val_trajectories)

In [5]:
# Initialising data loaders
BATCH_SIZE = 4
train_dataset = TensorDataset(train_ids)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(val_ids)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

### Training

In [6]:
LR = 1e-5
config = {"Learning Rate": LR,
          "Batch Size": BATCH_SIZE, 
          "Decimals": DECIMALS,
          "Lora Rank": LORA_RANK}
wandb.init(project="M2 Coursework", config=config)
wandb.watch(model, log="all", log_freq=10)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: at2128 (at2128-university-of-cambridge) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
train(model, train_loader, val_loader, learning_rate=LR, eval_interval=200, wandb=wandb)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Steps 2625:  39%|███▉      | 343/875 [02:54<05:38,  1.57it/s, loss=0.583]  